In [31]:
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from app import app

import math
import numpy as np
import pandas as pd
import plotly.offline as pyo
import plotly.express as px
import plotly.graph_objs as go
import dash as dash
import json
from datetime import datetime as dt
from datetime import timedelta
import re

from modules import module_vobcfault
from modules import module_commLoss
from views.ViewTrainmoveClass import ViewTrainmoveClass
from views.view_commLossListClass import ViewCommLossListClass


import util as util



In [33]:
def get_switch_time(start_date, end_date, s):
    start_date, end_date = util.date2str2(start_date,end_date)

    query = ("SELECT interval, switchId  "
             " from dlr_switch_move "
             " where interval > 0 and interval < 60 and loggedAt >= '{}' and loggedAt < '{}' and switchId = {}"
             " LIMIT 5000 ").format( start_date, end_date, s)
    
    df = util.run_query(query)

    return df

In [35]:
def get_switch_list(start_date, end_date):
    start_date, end_date = util.date2str2(start_date,end_date)

    query = ("SELECT switchId  "
             " from dlr_switch_move "
             " where interval > 0 and interval < 60 and loggedAt >= '{}' and loggedAt < '{}'"
             " group by switchId LIMIT 5000 ").format( start_date, end_date)
    
    L = util.run_query(query)

    return L

In [37]:
start_date = dt(2012, 1, 1)
end_date = dt(2020, 1, 1)
start_date, end_date  = util.date2str2(start_date, end_date )

In [39]:
def get_switch_count(start_date, end_date):
    start_date, end_date = util.date2str2(start_date,end_date)

    query = ("SELECT switchId, COUNT(*) as amt from dlr_switch_move" 
            " where interval >= 0 and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right' )" 
            " and loggedAt >= '{}' and loggedAt < '{}'" 
            " group by switchId").format( start_date, end_date)
    
    L = util.run_query(query)

    return L

In [40]:
def get_switch_filter_val(start_date, end_date, switchId, Limit):
    start_date, end_date = util.date2str2(start_date,end_date)
    Limit = int(Limit)
    query = ("SELECT interval, switchId from dlr_switch_move"
            " where interval > 0 and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right' )"
            " and switchId = '{}' and loggedAt >= '{}' and loggedAt < '{}'" 
            " order by interval desc LIMIT {}").format(switchId, start_date, end_date, Limit)
    
    L = util.run_query(query)
    if L is None or L.empty:
        return 0
    return L["interval"].min()

In [41]:
def get_avg(start_date, end_date, switchId, interval_max):
    start_date, end_date = util.date2str2(start_date,end_date) 
    query = ("SELECT AVG(interval) as int_Avg from dlr_switch_move"
            " where interval > 0 and interval < {}  and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right' )"
            " and switchId = '{}' and loggedAt >= '{}' and loggedAt < '{}'" 
            ).format(interval_max, switchId, start_date, end_date)
    
    L = util.run_query(query)
    if L is None or L.empty:
        return 0
    return L["int_Avg"].min()
  

In [42]:
def get_min(start_date, end_date, switchId, interval_max):
    start_date, end_date = util.date2str2(start_date,end_date)
    query = ("SELECT interval from dlr_switch_move"
            " where interval > 0 and interval < {}  and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right' )"
            " and switchId = '{}' and loggedAt >= '{}' and loggedAt < '{}'" 
            " order by interval ASC LIMIT 5").format(interval_max, switchId, start_date, end_date)

    L = util.run_query(query)
    if L is None or L.empty:
        return 0
    return L["interval"].min()

In [43]:
def get_d0(start_date, end_date, switchId, avg):
    start_date, end_date = util.date2str2(start_date,end_date)

    query = ("SELECT AVG(interval) as int_Avg from dlr_switch_move"
            " where interval > 0 and interval <= {}  and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right' )"
            " and switchId = '{}' and loggedAt >= '{}' and loggedAt < '{}'" 
            ).format(avg, switchId, start_date, end_date)

    L = util.run_query(query)
    if L is None or L.empty:
        return 0
    return L["int_Avg"].min()

In [44]:
def get_d1(start_date, end_date, switchId, avg, interval_max):
    start_date, end_date = util.date2str2(start_date,end_date)
    
    query = ("SELECT AVG(interval) as int_Avg from dlr_switch_move"
            " where interval > {} and interval < {}  and intervalDesc in ('Moving Time to Left' , 'Moving Time to Right')"
            " and switchId = '{}' and loggedAt >= '{}' and loggedAt < '{}'" 
            ).format(avg, interval_max, switchId, start_date, end_date)

    L = util.run_query(query)
    if L is None or L.empty:
        return 0
    return L["int_Avg"].max()

In [135]:
df1 = get_switch_count(start_date, end_date)
df1["amt"] = df1["amt"].div(1000)

df1 

,switchId,amt
0,101,350.077
1,104,1.475
2,105,1.475
3,106,2.252
4,107,2.251
...,...,...
129,410,1.187
130,411,8.881
131,412,8.881
132,413,132.290


In [117]:
%%timeit

df2 = df1
df2["max"] = 0.0
for ind in df2.index:
    df2["max"][ind] = get_switch_filter_val(start_date, end_date, df2["switchId"][ind], df2["amt"][ind])
pd.set_option('precision', 5)
df2 


3.46 s ± 187 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [125]:
%%timeit
df2 = df1
filter_series = []
df2["max"] = 0.0
for index, row in df2.iterrows():
    filter_series.append(get_switch_filter_val(start_date, end_date, row["switchId"], row["amt"]))
df2["max"] = filter_series

3.28 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [136]:

df2 = df1
df2['max'] = df2.apply(lambda row: get_switch_filter_val(start_date, end_date, row["switchId"], row["amt"]), axis=1)
df2

,switchId,amt,max
0,101,350.077,3.34
1,104,1.475,4.18
2,105,1.475,4.18
3,106,2.252,54.34
4,107,2.251,54.34
...,...,...,...
129,410,1.187,6.13
130,411,8.881,4.04
131,412,8.881,4.04
132,413,132.290,3.97


In [131]:
%%timeit
df3 = df2
df3["d1"] = 0.0
df3["avg"] = 0.0
df3["d0"] = 0.0
df3["min"] = 0.0

for ind in df3.index:
    df3["avg"][ind] = get_avg(start_date, end_date, df3["switchId"][ind], df3["max"][ind])
    df3["min"][ind] = get_min(start_date, end_date, df3["switchId"][ind], df3["max"][ind])
    df3["d0"][ind] = get_d0(start_date,end_date, df3["switchId"][ind], df3["avg"][ind])
    df3["d1"][ind] = get_d1(start_date,end_date, df3["switchId"][ind], df3["avg"][ind], df3["max"][ind])

pd.set_option('precision', 5)
df3 

7.55 s ± 130 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:

df3 = df2


df3["avg"] = df3.apply(lambda row: get_avg(start_date, end_date, row["switchId"], row["max"]), axis=1)
df3["min"] = df3.apply(lambda row: get_min(start_date, end_date, row["switchId"], row["max"]), axis=1)
df3["d0"] = df3.apply(lambda row: get_d0(start_date,end_date, row["switchId"], row["avg"]), axis=1)
df3["d1"] = df3.apply(lambda row: get_d1(start_date,end_date, row["switchId"], row["avg"], row["max"]), axis=1)


df3 

,switchId,amt,max,avg,min,d0,d1
0,101,350.077,3.34,3.13461,2.08,3.12967,3.15979
1,104,1.475,4.18,3.13592,2.08,3.12915,3.17211
2,105,1.475,4.18,3.13663,2.08,3.12915,3.17639
3,106,2.252,54.34,3.15494,2.09,3.13067,3.45440
4,107,2.251,54.34,3.15494,2.09,3.13066,3.45440
...,...,...,...,...,...,...,...
129,410,1.187,6.13,3.81547,3.06,3.07043,3.83022
130,411,8.881,4.04,3.81822,3.06,3.06590,3.83063
131,412,8.881,4.04,3.81822,3.06,3.06590,3.83063
132,413,132.290,3.97,3.81095,1.53,3.06299,3.82959


In [ ]:
df3 = df2


df3["avg"] = df3.apply(lambda row: get_avg(start_date, end_date, row["switchId"], row["max"]), axis=1)
df3["min"] = df3.apply(lambda row: get_min(start_date, end_date, row["switchId"], row["max"]), axis=1)
df3["d0"] = df3.apply(lambda row: get_d0(start_date,end_date, row["switchId"], row["avg"]), axis=1)
df3["d1"] = df3.apply(lambda row: get_d1(start_date,end_date, row["switchId"], row["avg"], row["max"]), axis=1)


df3 

In [ ]:
df3 = df2


df3["avg"] = df3.apply(lambda row: get_avg(start_date, end_date, row["switchId"], row["max"]), axis=1)
df3["min"] = df3.apply(lambda row: get_min(start_date, end_date, row["switchId"], row["max"]), axis=1)
df3["d0"] = df3.apply(lambda row: get_d0(start_date,end_date, row["switchId"], row["avg"]), axis=1)
df3["d1"] = df3.apply(lambda row: get_d1(start_date,end_date, row["switchId"], row["avg"], row["max"]), axis=1)


df3 

In [69]:
Lof = df3['min'].tolist()
d0 = df3['d0'].tolist()
avg = df3['avg'].tolist()
d1 = df3['d1'].tolist()
Uof = df3['max'].tolist()

In [70]:
df3['max'][3]

54.34000015258789

In [97]:
fig = go.Figure()

for ind in df3.index:
    data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    fig.add_trace(go.Box(
                    y= data,
                    x = [df3["switchId"][ind]],
                    q1=[df3['d0'][ind]], 
                    median=[df3['avg'][ind]],
                    q3=[df3['d1'][ind]], 
                    #lowerfence=[df3['min'][ind]],
                    #upperfence=[df3['max'][ind]]
                    ))

fig.show()

In [89]:
fig = go.Figure()
data = [1, 2, 3, 4, 5, 6, 7, 8]
fig.add_trace(go.Box(y = data,
                q1=d0, 
                median=avg,
                q3=d1, 
                lowerfence=Lof,
                upperfence=Uof
                    ))

fig.show()

In [104]:
fig = go.Figure()


data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
fig.add_trace(go.Box(
                    y= data,
                    #x = [df3["switchId"][0]],
                    q1=[df3['d0'][0]], 
                    median=[df3['avg'][0]],
                    q3=[df3['d1'][0]], 
                    lowerfence=[df3['min'][0]],
                    upperfence=[df3['max'][0]]
                    ))

fig.show()

In [105]:
fig = go.Figure()


data = [1, 2, 3, 4, 5, 6, 7, 8, 9]
fig.add_trace(go.Box(
                    y= data,
                    #x = [df3["switchId"][0]],
                    q1=[1.5], 
                    median=[2],
                    q3=[2.3], 
                    lowerfence=[1],
                    upperfence=[3]
                    ))

fig.show()

In [108]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Box(y=[
        [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ],
        [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ],
        [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ]
      ], name="Precompiled Quartiles"))

fig.update_traces(q1=[ 1, 2, 3 ], median=[ 4, 5, 6 ],
                  q3=[ 7, 8, 9 ], lowerfence=[-1, 0, 1],
                  upperfence=[5, 6, 7], mean=[ 2.2, 2.8, 3.2 ],
                  sd=[ 0.2, 0.4, 0.6 ], notchspan=[ 0.2, 0.4, 0.6 ] )

fig.show()